In [27]:
import gmaps
import pandas as pd
from api_keys import g_key
import requests
import json

In [28]:
city_df = pd.read_csv('../Weather Py/city_data.csv')
city_df=city_df.dropna()
city_df.head()

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,0,Hithadhoo,84.45,61,4,8.14,-0.60,73.08
1,1,Ponta do Sol,61.27,59,40,23.04,32.67,-17.10
2,2,Itarema,81.57,83,100,4.29,-2.92,-39.92
3,3,Sturgeon Falls,13.03,51,1,11.41,46.36,-79.93
4,4,Ushuaia,51.80,76,75,11.41,-54.80,-68.30


In [29]:

# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]].astype(float)

# convert to float
humidity = city_df["Humidity"].astype(float)

In [30]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
#Finding your ideal weather condition
ideal_df=city_df

#ideal weather temperaute 
ideal_df=ideal_df.loc[(ideal_df['Temperature']<85)&(ideal_df['Temperature']>75)]

#ideal wind speed
ideal_df=ideal_df.loc[(ideal_df['Wind Speed']<10)]
                      
#ideal cloudiness
ideal_df=ideal_df.loc[(ideal_df['Cloudiness']==0)]

ideal_df=ideal_df.dropna()

ideal_df

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
86,86,Alice Springs,80.60,11,0,8.05,-23.70,133.88
100,100,Colesberg,79.57,34,0,5.39,-30.72,25.10
134,134,São João da Barra,77.00,88,0,8.81,-21.64,-41.05
291,291,Vila Velha,75.99,88,0,3.36,-20.33,-40.29
389,389,Awjilah,83.01,12,0,1.68,29.11,21.29
509,509,Sabha,83.01,8,0,5.91,27.04,14.43
527,527,Aswān,78.80,29,0,9.17,24.09,32.91


In [33]:
#find the first hotel for each city located within 5000 meters of your coordinates
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in ideal_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        ideal_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 86: Alice Springs.
Closest hotel is Desert Palms Alice Springs.
------------
Retrieving Results for Index 100: Colesberg.
Closest hotel is Bhelekazi Guesthouse.
------------
Retrieving Results for Index 134: São João da Barra.
Closest hotel is Pousada Porto De Canoas.
------------
Retrieving Results for Index 291: Vila Velha.
Closest hotel is Hotel Vitória Palace.
------------
Retrieving Results for Index 389: Awjilah.
Missing field/result... skipping.
------------
Retrieving Results for Index 509: Sabha.
Closest hotel is محطة ام الارانب -سبها مواصلات.
------------
Retrieving Results for Index 527: Aswān.
Closest hotel is Nuba Nile.
------------


In [37]:
ideal_df=ideal_df.dropna()

ideal_df

,Unnamed: 0,City,Temperature,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
86,86,Alice Springs,80.60,11,0,8.05,-23.70,133.88,Desert Palms Alice Springs
100,100,Colesberg,79.57,34,0,5.39,-30.72,25.10,Bhelekazi Guesthouse
134,134,São João da Barra,77.00,88,0,8.81,-21.64,-41.05,Pousada Porto De Canoas
291,291,Vila Velha,75.99,88,0,3.36,-20.33,-40.29,Hotel Vitória Palace
509,509,Sabha,83.01,8,0,5.91,27.04,14.43,محطة ام الارانب -سبها مواصلات
527,527,Aswān,78.80,29,0,9.17,24.09,32.91,Nuba Nile


In [47]:
# Assign the marker layer to a variable
locations = ideal_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))